# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [516]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [517]:
import polars as pl 
from google.cloud import bigquery
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import sys 
import os 


# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

In [518]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [ ]:
# Create client object
client = bigquery.Client()

In [ ]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [ ]:
run_query = client.query(QUERY)

In [ ]:
df = pl.from_arrow(run_query.to_arrow())

In [ ]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [ ]:
df.shape

(49512, 13)

In [ ]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [ ]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501533508,"""7227604725376239031""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",null,"""Android Infant Short Sleeve Te…","""Home/Apparel/Kid's/Kid's-Infan…"


In [ ]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [ ]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [ ]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [ ]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [ ]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))
df_cleaned

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,i32,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Take Charge Shor…","""Apparel"""


# Encoding the df

In [ ]:
encoded_df = encoding(df)
encoded_df

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklea

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName
str,i32,i32,i32,i32,i32,i32,i32,i32
"""2219384770970157334""",79,67,5,6,0,8,0,92
"""2219384770970157334""",79,67,5,6,0,8,0,49
"""2219384770970157334""",79,67,5,6,0,8,0,202
"""2219384770970157334""",79,67,5,6,0,8,0,126
"""2219384770970157334""",79,67,5,6,0,8,0,203
…,…,…,…,…,…,…,…,…
"""7483600664917507409""",94,37,2,4,0,0,1,8
"""7483600664917507409""",94,37,2,4,0,0,1,21
"""7483600664917507409""",94,37,2,4,0,0,1,23


In [ ]:
user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('city'), 
    pl.col('browser'),pl.col('operatingSystem'),pl.col('deviceCategory'),
    pl.col('source'),pl.col('transactionId'),pl.col('v2ProductName')
)
user_profiles

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName
str,list[i32],list[i32],list[i32],list[i32],list[i32],list[i32],list[i32],list[i32]
"""8330772482336842742""","[94, 94, … 94]","[30, 30, … 30]","[2, 2, … 2]","[6, 6, … 6]","[0, 0, … 0]","[8, 8, … 8]","[0, 0, … 0]","[97, 17, … 18]"
"""074039193550958685""","[28, 28, … 28]","[67, 67, … 67]","[2, 2, … 2]","[6, 6, … 6]","[0, 0, … 0]","[20, 20, … 20]","[0, 0, … 0]","[224, 2, … 223]"
"""3456285538294788190""","[94, 94, … 94]","[22, 22, … 22]","[10, 10, … 10]","[8, 8, … 8]","[1, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[207, 86, … 4]"
"""7064330076100499572""","[94, 94, … 94]","[67, 67, … 67]","[10, 10, … 10]","[8, 8, … 8]","[2, 2, … 2]","[8, 8, … 8]","[0, 0, … 0]","[224, 2, … 223]"
"""1807640794779781875""","[16, 16, … 16]","[67, 67, … 67]","[10, 10, … 10]","[8, 8, … 8]","[2, 2, … 2]","[8, 8, … 8]","[0, 0, … 0]","[12, 182, … 194]"
…,…,…,…,…,…,…,…,…
"""2508529781333336867""",[94],[67],[10],[8],[1],[0],[0],[119]
"""7369274377505861716""","[94, 94, … 94]","[67, 67, … 67]","[5, 5, … 5]","[6, 6, … 6]","[0, 0, … 0]","[20, 20, … 20]","[0, 0, … 0]","[224, 2, … 223]"
"""8764003936075582308""","[94, 94, … 94]","[37, 37, … 37]","[2, 2, … 2]","[4, 4, … 4]","[0, 0, … 0]","[8, 8, … 8]","[0, 0, … 0]","[12, 182, … 105]"


In [ ]:
user_profiles = user_profiles.filter(pl.col('fullVisitorId') != '0345672699449577691')

In [ ]:
user_profiles_no_id = user_profiles.drop('fullVisitorId')
user_profiles_no_id = user_profiles_no_id.to_numpy()
user_profiles_no_id

array([[array([94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94]),
        array([30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]),
        array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), ...,
        array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([ 97,  17, 115, 230,  24, 141, 231,  25,  19,  22,  21,  18])],
       [array([28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28]),
        array([67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67]),
        array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), ...,
        array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([224,   2, 233, 225, 230, 231, 229, 228, 227, 235, 236, 223])],
       [array([94, 94, 94, 94, 94]), array([22, 22, 22, 22, 22]),
        array([10, 10, 10, 10, 10]), ..., array([0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0]), array([207,  86, 125,  64,   4])],
       ...,
       [array([94, 94, 94, 94, 94

In [ ]:
active_users = encoded_df.drop('fullVisitorId')
active_users = active_users.to_numpy()
active_users

array([[ 79,  67,   5, ...,   8,   0,  92],
       [ 79,  67,   5, ...,   8,   0,  49],
       [ 79,  67,   5, ...,   8,   0, 202],
       ...,
       [ 94,  37,   2, ...,   0,   1,  23],
       [ 94,  37,   2, ...,   0,   1, 156],
       [ 94,  37,   2, ...,   0,   0,  95]])

In [ ]:
test_users = [2,	9,	0,	1,	0,	0,	1,	26]
test_users

[2, 9, 0, 1, 0, 0, 1, 26]

In [ ]:
active_users[-1]

array([94, 37,  2,  4,  0,  0,  0, 95])

In [ ]:
similarities = calculate_similarity(user_profiles, user_profiles_no_id, test_users)
similarities

{'8330772482336842742': 0.5705597593978505,
 '074039193550958685': 0.9394544618310685,
 '3456285538294788190': 0.6607336141877771,
 '7064330076100499572': 0.8943936450221299,
 '1807640794779781875': 0.9201354478747985,
 '7481813821985673901': 0.9385872850296185,
 '0280589015308186459': 0.7782638982377438,
 '7137691471308695271': 0.8671354132438217,
 '4767330263280425166': 0.9403602456766317,
 '9776744955382733824': 0.7120750573347903,
 '9540334486012909236': 0.7355814735714606,
 '8408186041897916606': 0.6115050029107175,
 '5647373324104834148': 0.877663307535638,
 '8353844990840662374': 0.7774925942341804,
 '4613632710734023206': 0.7029565418090353,
 '9352400691264369031': 0.5312305178573475,
 '2174668406699092948': 0.9114075456176388,
 '9121152115784735083': 0.8172386392095584,
 '2142324270327336270': 0.845269187660974,
 '5273246661244498767': 0.9353548034287927,
 '742908355291567030': 0.871952292293915,
 '8158726673419204452': 0.7668909881307245,
 '11577526465501235': 0.7609505467778

In [ ]:
sorted_dict = sorted(similarities.items(), key=lambda x: x[1])
sorted_dict[-5:]

[('24486799598615247', 0.9910547429548338),
 ('8319892827536314987', 0.9940268445351574),
 ('9762695135266003658', 0.9969107467745082),
 ('7958854055537008406', 0.9983318428069594),
 ('3998764704348759017', 0.9988188806470808)]

In [ ]:
actual_bought = df.filter((pl.col('fullVisitorId') == '0345672699449577691') & (pl.col('transactionId') != 'null')).select(pl.col('v2ProductName')).to_series().to_list()
actual_bought

['Google Blackout Cap',
 "Google Women's Vintage Hero Tee Black",
 "Google Men's Long & Lean Tee Charcoal",
 "Google Men's Vintage Badge Tee Black",
 'Google Blackout Cap',
 "Google Women's Vintage Hero Tee Black",
 "Google Men's Long & Lean Tee Charcoal",
 "Google Men's Vintage Badge Tee Black"]

In [ ]:
recommendation(df,sorted_dict)

['Waze Mood Original Window Decal',
 "YouTube Men's Short Sleeve Hero Tee Charcoal",
 'Keyboard DOT Sticker',
 'Google Vintage Henley Grey/Black',
 'Google Tri-blend Hoodie Grey']

In [ ]:
rec = recommendation(df,sorted_dict)

In [ ]:
# actual_bought=set(actual_bought)
# rec=set(rec)

In [ ]:
actual_bought

{'Google Blackout Cap',
 "Google Men's Long & Lean Tee Charcoal",
 "Google Men's Vintage Badge Tee Black",
 "Google Women's Vintage Hero Tee Black"}

In [ ]:
rec

{"Google Men's Colorblock Tee White/Heather",
 'Google Tri-blend Hoodie Grey',
 'Keyboard DOT Sticker',
 'Waze Mood Original Window Decal',
 "YouTube Men's Short Sleeve Hero Tee Charcoal"}

In [ ]:
set(actual_bought & rec)

set()

In [ ]:
precision(actual_bought, rec, 3)

TypeError: 'set' object is not subscriptable

In [ ]:
df.write_csv("../data/cleaned_google_analytics.csv")

### Data Pre-processing Pipeline

In [ ]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df = encoding(df)
    
    user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
        pl.col('country'), pl.col('browser'),
        pl.col('transactionId'), pl.col('v2ProductName')
    )
    user_profiles_no_id = user_profiles.drop('fullVisitorId')
    user_profiles_no_id = user_profiles_no_id.to_numpy()
    active_users = encoded_df.drop('fullVisitorId')
    active_users = active_users.to_numpy()

    similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users)
    sorted_dict = sorted(similarities.items(), key=lambda x: x[1])

    
    df.write_csv("../data/cleaned_google_analytics.csv")
    return recommendation(df,sorted_dict) 

In [ ]:
preprocessing_pipeline(df)

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklea

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

# Conclusions 
- 